# CSK constellation optimization

* $\alpha \in \mathbb{R}^{N\times 3}$ is the set of input points, N vectors of length 3 (r,g,b)
* $H \in \mathbb{R}^{3\times 3}$ is the transformation matrix


The problem to solve is:
$$
\begin{aligned}
\max \min_{\alpha} \quad & ||\alpha H_{i,:} - \alpha H_{j,:}||_{2}^{2}, i=1...N, j=1...N, i\ne j\\
\textrm{s.t.} \quad & \sum_{j=1}^{3}\alpha_{i,j} \le 1, i=1...N\\
  &\alpha >= 0    \\
\end{aligned}
$$


This $\max \min$ problem can be rewritten as:

$$
\begin{aligned}
\max_{\alpha} \quad & Z\\ 
\textrm{s.t.} \quad & \sum_{j=1}^{3}\alpha_{i,j} \le 1, i=1...N\\
  &\alpha >= 0    \\
  & Z \le ||\alpha H_{1,:} - \alpha H_{2,:}||_{2}^{2}\\
  & Z \le ||\alpha H_{1,:} - \alpha H_{3,:}||_{2}^{2}\\
  & \vdots
\end{aligned}
$$

---
Install and import required packages

In [ ]:
using Pkg
Pkg.add("JuMP")        # to express optimization problems
Pkg.add("Ipopt")       # solver
Pkg.add("PlotlyJS")    # for plotting
Pkg.add("DataFrames")

Check packages versions (for reference)

In [ ]:

Pkg.status()

Status `~/.julia/environments/v1.9/Project.toml`
  [a93c6f00] DataFrames v1.6.1
  [7073ff75] IJulia v1.24.2
  [b6b21f68] Ipopt v1.4.2
  [4076af6c] JuMP v1.15.1
  [f0f68f2c] PlotlyJS v0.18.10
  [0f1e0344] WebIO v0.8.21


In [ ]:
using JuMP
using Ipopt
using PlotlyJS
using DataFrames


---
## Implementation
Number of points in the constellation

In [ ]:
N = 16
M = 2 

2

Define an empty model and assign the solver.

**TODO: experiment with different NLP [solvers](https://jump.dev/JuMP.jl/stable/installation/#Supported-solvers)**

In [ ]:

model = Model(Ipopt.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Ipopt

Define the variable $\alpha$. It is very convenient that it can be defined as a $N\times 3$ matrix.

In [ ]:
@variable(model, ɑ[1:N, 1:M]  >= 0.0)


16×2 Matrix{VariableRef}:
 ɑ[1,1]   ɑ[1,2]
 ɑ[2,1]   ɑ[2,2]
 ɑ[3,1]   ɑ[3,2]
 ɑ[4,1]   ɑ[4,2]
 ɑ[5,1]   ɑ[5,2]
 ɑ[6,1]   ɑ[6,2]
 ɑ[7,1]   ɑ[7,2]
 ɑ[8,1]   ɑ[8,2]
 ɑ[9,1]   ɑ[9,2]
 ɑ[10,1]  ɑ[10,2]
 ɑ[11,1]  ɑ[11,2]
 ɑ[12,1]  ɑ[12,2]
 ɑ[13,1]  ɑ[13,2]
 ɑ[14,1]  ɑ[14,2]
 ɑ[15,1]  ɑ[15,2]
 ɑ[16,1]  ɑ[16,2]

In [ ]:
@constraint(model, sum(ɑ, dims=2) .- 1 .== 0)


16×1 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 ɑ[1,1] + ɑ[1,2] = 1
 ɑ[2,1] + ɑ[2,2] = 1
 ɑ[3,1] + ɑ[3,2] = 1
 ɑ[4,1] + ɑ[4,2] = 1
 ɑ[5,1] + ɑ[5,2] = 1
 ɑ[6,1] + ɑ[6,2] = 1
 ɑ[7,1] + ɑ[7,2] = 1
 ɑ[8,1] + ɑ[8,2] = 1
 ɑ[9,1] + ɑ[9,2] = 1
 ɑ[10,1] + ɑ[10,2] = 1
 ɑ[11,1] + ɑ[11,2] = 1
 ɑ[12,1] + ɑ[12,2] = 1
 ɑ[13,1] + ɑ[13,2] = 1
 ɑ[14,1] + ɑ[14,2] = 1
 ɑ[15,1] + ɑ[15,2] = 1
 ɑ[16,1] + ɑ[16,2] = 1

**TODO: try with different $H$ matrices**

In [ ]:
H = 55.3e-3*[1 0.042  0.030; 0.194  0.665  0.277;0.009 0.084 0.421]


3×3 Matrix{Float64}:
 0.0553     0.0023226  0.001659
 0.0107282  0.0367745  0.0153181
 0.0004977  0.0046452  0.0232813

**TODO: Explore with different norms. Using the Euclidean norm results in  a solver error.**

In [ ]:
manhattan(x,y) = sum((abs(x[i]-y[i]) for i in 1:M))
euclid(x,y) = sqrt(sum((x[i]-y[i])^2 for i in 1:M))
# norm = euclid # for the time-being only the Manhattan norm is working
norm = manhattan # for the time-being only the Manhattan norm is working


manhattan (generic function with 1 method)

In [ ]:
# ɑH = ɑ*H  # define output constellation
ɑH = ɑ  # define output constellation

# calculate pairwise distances
dist_ = [norm(ɑH[i,:], ɑH[j,:]) for i in 1:N for j in 1:N if i != j]
Nd = size(dist_)[1]
@expression(model, dist[i=1:Nd], dist_[i])



240-element Vector{NonlinearExpr}:
 abs(ɑ[1,1] - ɑ[2,1]) + abs(ɑ[1,2] - ɑ[2,2])
 abs(ɑ[1,1] - ɑ[3,1]) + abs(ɑ[1,2] - ɑ[3,2])
 abs(ɑ[1,1] - ɑ[4,1]) + abs(ɑ[1,2] - ɑ[4,2])
 abs(ɑ[1,1] - ɑ[5,1]) + abs(ɑ[1,2] - ɑ[5,2])
 abs(ɑ[1,1] - ɑ[6,1]) + abs(ɑ[1,2] - ɑ[6,2])
 abs(ɑ[1,1] - ɑ[7,1]) + abs(ɑ[1,2] - ɑ[7,2])
 abs(ɑ[1,1] - ɑ[8,1]) + abs(ɑ[1,2] - ɑ[8,2])
 abs(ɑ[1,1] - ɑ[9,1]) + abs(ɑ[1,2] - ɑ[9,2])
 abs(ɑ[1,1] - ɑ[10,1]) + abs(ɑ[1,2] - ɑ[10,2])
 abs(ɑ[1,1] - ɑ[11,1]) + abs(ɑ[1,2] - ɑ[11,2])
 abs(ɑ[1,1] - ɑ[12,1]) + abs(ɑ[1,2] - ɑ[12,2])
 abs(ɑ[1,1] - ɑ[13,1]) + abs(ɑ[1,2] - ɑ[13,2])
 abs(ɑ[1,1] - ɑ[14,1]) + abs(ɑ[1,2] - ɑ[14,2])
 ⋮
 abs(ɑ[16,1] - ɑ[4,1]) + abs(ɑ[16,2] - ɑ[4,2])
 abs(ɑ[16,1] - ɑ[5,1]) + abs(ɑ[16,2] - ɑ[5,2])
 abs(ɑ[16,1] - ɑ[6,1]) + abs(ɑ[16,2] - ɑ[6,2])
 abs(ɑ[16,1] - ɑ[7,1]) + abs(ɑ[16,2] - ɑ[7,2])
 abs(ɑ[16,1] - ɑ[8,1]) + abs(ɑ[16,2] - ɑ[8,2])
 abs(ɑ[16,1] - ɑ[9,1]) + abs(ɑ[16,2] - ɑ[9,2])
 abs(ɑ[16,1] - ɑ[10,1]) + abs(ɑ[16,2] - ɑ[10,2])
 abs(ɑ[16,1] - ɑ[11,1]) + abs(ɑ[16,2

There are two different ways (at least) to formulate the optimization problem
* Directly formulating the $\max \min$ problem
* Rewriting it as a $\max$ problem with a slack variable and additional constraints

Only the latter one is working

**TODO: investigate why**

In [ ]:
approx = true
if approx
	# uses the rewriting of the maxmin
	@variable(model, Z >= 0)
	@constraint(model, Z .<= dist)
	@objective(model, Max, Z)
else
	# this approch is currently not working. TODO: investigate
	f(args...) = min(args...)
	@objective(model, Max, f(dist...))
end

Z

In [ ]:
optimize!(model)

This is Ipopt version 3.14.13, running with linear solver MUMPS 5.6.1.

Number of nonzeros in equality constraint Jacobian...:       32
Number of nonzeros in inequality constraint Jacobian.:     1200
Number of nonzeros in Lagrangian Hessian.............:     1440

Total number of variables............................:       33
                     variables with only lower bounds:       33
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       16
Total number of inequality constraints...............:      240
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:      240

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  9.9999900e-03 9.80e-01 9.88e-01  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

In [ ]:
solution_summary(model)

* Solver : Ipopt

* Status
  Result count       : 1
  Termination status : LOCALLY_SOLVED
  Message from the solver:
  "Solve_Succeeded"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 1.33333e-01
  Dual objective value : 1.33334e-01

* Work counters
  Solve time (sec)   : 2.92902e-01


Get the results

In [ ]:
input = value.(ɑ)
# output = input * H

16×2 Matrix{Float64}:
 0.333333    0.666667
 0.266667    0.733333
 0.0666667   0.933333
 0.6         0.4
 0.466667    0.533333
 1.0         8.79421e-9
 0.866667    0.133333
 0.733333    0.266667
 8.79421e-9  1.0
 0.8         0.2
 0.933333    0.0666667
 0.533333    0.466667
 0.133333    0.866667
 0.2         0.8
 0.4         0.6
 0.666667    0.333333

## Plot results

Plot input constellation

In [ ]:
df_i = DataFrame(input, :auto)
# plot(df_i, x=:x1, y=:x2, z=:x3, type="scatter3d", mode="markers")

# Assuming you have a DataFrame df_i with columns x1 and x2
plot(df_i, x=:x1, y=:x2, type="scatter", mode="markers")

data: [
  "scatter3d with fields mode, type, x, xaxis, y, yaxis, and z"
]

layout: "layout with fields legend, margin, template, xaxis, and yaxis"

plot output constellation

In [ ]:
df_out = DataFrame(output, :auto)
plot(df_out, x=:x1, y=:x2, z=:x3, type="scatter3d", mode="markers")

data: [
  "scatter3d with fields mode, type, x, xaxis, y, yaxis, and z"
]

layout: "layout with fields legend, margin, template, xaxis, and yaxis"

Verify Euclidean distance for input and output points:

In [ ]:
euclid_norm  = [euclid(output[i,:], output[j,:]) for i in 1:N for j in 1:N if i != j];
min(euclid_norm...)

0.006540732663466468

In [ ]:
plot(euclid_norm)

data: [
  "scatter with fields type, x, and y"
]

layout: "layout with fields margin and template"

In [ ]:
plot(DataFrame([euclid_norm], :auto), x=:x1, kind="histogram")

data: [
  "histogram with fields type, x, xaxis, and yaxis"
]

layout: "layout with fields legend, margin, template, and xaxis"

In [ ]:
euclid_norm  = [euclid(input[i,:], input[j,:]) for i in 1:N for j in 1:N if i != j];
min(euclid_norm...)

0.09428090249918696